# Декораторы II

In [2]:
import functools

In [3]:
def validate_user_id(func):
    @functools.wraps(func)
    def wrapped(*args, **kwargs):
        if 'user_id' not in kwargs or kwargs['user_id'] == 0:
            print('user_id is wrong')
            return

        print('user_id is correct')
        return func(*args, **kwargs)
    
    return wrapped


def log_access(func):
    @functools.wraps(func)
    def wrapped(*args, **kwargs):
        print('{} was called'.format(func.__name__))
        return func(*args, **kwargs)
    return wrapped


@log_access
@validate_user_id
def feedback(user_id, message):
    print('Feedback received')
    

# feedback = log_access(validate_user_id(feedback))

data = {
    'message': 'Your app is awesome!',
    'user_id': 42
}

feedback(**data)

feedback was called
user_id is correct
Feedback received


In [4]:
def bold(func):
    def wrapped():
        return "<b>" + func() + "</b>"
    return wrapped


def italic(func):
    def wrapped():
        return "<i>" + func() + "</i>"
    return wrapped


@bold
@italic
def hello():
    return "hello world"


# hello = bold(italic(hello))


print(hello())

<b><i>hello world</i></b>


## Кто пробовал писать декораторы после 3 лекции?

In [7]:
import functools
import json


def to_json(func):
    @functools.wraps(func)
    def wrapped(*args, **kwargs):
        result = func(*args, **kwargs)
        return json.dumps(result)
    
    return wrapped



@to_json
def feedback(user_id, message):
    return {
        'user_id': 10,
        'result': 'Success'
    }


print(feedback(10, 'Hello'))
    

{"user_id": 10, "result": "Success"}


# Итераторы


In [126]:
iterator = iter([1, 2, 3])

In [127]:
print(next(iterator))

1


In [14]:
class EvenIterator:
    def __init__(self, start, end):
        self.current = start
        self.end = end
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.current > self.end:
            raise StopIteration

        self.current += 2
        return self.current - 2
    
    
for num in EvenIterator(1, 10):
    print(num)
        

1
3
5
7
9


In [124]:
class IndexIterable:
    def __init__(self, obj):
        self.obj = obj
        
    def __getitem__(self, index):
        return self.obj[index]


for letter in IndexIterable('123'):
    print(letter)

    
# Какой еще объект может быть вместо строки?

1
2
3


In [18]:
import collections



print(isinstance(EvenIterator(1, 10), collections.Iterable))
print(isinstance(IndexIterable('123'), collections.Iterable))


True
False


# Асинхронные итераторы

In [19]:
import asyncio


class Ranger:
    def __init__(self, limit):
        self.limit = limit

    async def __aiter__(self):
        return self

    async def __anext__(self):
        await asyncio.sleep(1)
        if self.limit == 0:
            raise StopAsyncIteration
            
        self.limit -= 1
        return self.limit + 1
       

async def main():
    async for val in Ranger(10):
        print(val)

    
loop = asyncio.get_event_loop()
loop.run_until_complete(main())

10
9
8
7
6
5
4
3
2
1


# Генераторы

### Что такое генераторы и зачем они нужны?

In [40]:
def fibonacci(number):
    a = b = 1
    for _ in range(number):
        yield a
        a, b = b, a + b
        

# for num in fibonacci(100):
#     print(num)

In [162]:
def list_generator(list_obj):
    for item in list_obj:
        yield item
        print('After yielding {}'.format(item))


generator = list_generator([1, 2, 3])

In [168]:
# print(next(generator))

In [113]:
def accumulator():
    total = 0
    while True:
        value = yield total
        if not value:
            break

        total += value
        

g = accumulator()

# next(g)

# print(g.send(1))
# print(g.send(2))
# print(g.send(3))

# next(g)

In [123]:
def catcher():
    try:
        while True:
            yield 1

    except ValueError:
        while True:
            yield 2
    

g = catcher()

next(g)

g.throw(ValueError)

next(g)

2

# Асинхронные генераторы

### Планируются в 3.6 — [PEP 525](https://www.python.org/dev/peps/pep-0525/)

# Списочные выражения

In [20]:
import random


def get_random_list(length):
    random_list = []
    for _ in range(length):
        random_list.append(random.random())
    
    return random_list


print(get_random_list(10))

[0.14713373314738265, 0.5324055465493841, 0.06221239659027611, 0.7973969979541332, 0.6148950393495459, 0.883850915590419, 0.24875371942098568, 0.45141993036441475, 0.8756595901494661, 0.4606344701674665]


In [21]:
import random


def get_random_list(length):
    return [random.random()] * length

print(get_random_list(10))

[0.9590157719391453, 0.9590157719391453, 0.9590157719391453, 0.9590157719391453, 0.9590157719391453, 0.9590157719391453, 0.9590157719391453, 0.9590157719391453, 0.9590157719391453, 0.9590157719391453]


In [22]:
import random


def get_random_list(length):
    return [random.random() for _ in range(length)]

print(get_random_list(10))

[0.40972639635162245, 0.6491307022273962, 0.5999701597755943, 0.5382666524781086, 0.16990269903470256, 0.42025314098698396, 0.7110612779170754, 0.9466848672277495, 0.11573885738132172, 0.6080809888416031]


In [23]:
[str(num) for num in range(10)]

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [24]:
[str(num) for num in range(10) if num % 2]

['1', '3', '5', '7', '9']

In [25]:
[str(num) if num % 2 else num for num in range(10)]

[0, '1', 2, '3', 4, '5', 6, '7', 8, '9']

In [26]:
{num % 10 for num in range(100)}

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [27]:
{num: num % 3 for num in range(20)}

{0: 0,
 1: 1,
 2: 2,
 3: 0,
 4: 1,
 5: 2,
 6: 0,
 7: 1,
 8: 2,
 9: 0,
 10: 1,
 11: 2,
 12: 0,
 13: 1,
 14: 2,
 15: 0,
 16: 1,
 17: 2,
 18: 0,
 19: 1}

In [28]:
dict((x, x ** 2) for x in range(4))

{0: 0, 1: 1, 2: 4, 3: 9}

In [29]:
(num ** 2 for num in range(10))

<generator object <genexpr> at 0x101eea410>

In [30]:
[i * j for i in range(5) for j in range(5, 10)]

[0,
 0,
 0,
 0,
 0,
 5,
 6,
 7,
 8,
 9,
 10,
 12,
 14,
 16,
 18,
 15,
 18,
 21,
 24,
 27,
 20,
 24,
 28,
 32,
 36]

In [31]:
[[x for x in range(5)] for _ in range(5)]

[[0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4]]

In [35]:
[sorted(x) for x in [[2, 1], [4, 3], [0, 1]]]

[[1, 2], [3, 4], [0, 1]]

In [34]:
[print(x) for x in (1, 2, 3)]

1
2
3


[None, None, None]

In [36]:
a = [x ** 2 for x in range(1, 5)]
b = [x ** 3 for x in range(1, 5)]

list(zip(a, b))

[(1, 1), (4, 8), (9, 27), (16, 64)]

In [37]:
list(zip(a * 2, b))

[(1, 1), (4, 8), (9, 27), (16, 64)]

# Асинхронные comprehensions

### Планируются в 3.6 — [PEP 530](https://www.python.org/dev/peps/pep-0530/)

In [44]:
# [i async for i in agen()]

# Контекстные менеджеры

In [38]:
with open('access_log.log', 'a') as f:
    f.write('New Access\n')


In [39]:
class open_file:
    def __init__(self, filename, mode):
        self.f = open(filename, mode)
    
    def __enter__(self):
        return self.f
    
    def __exit__(self, *args):
        self.f.close()
        
        
with open_file('test.log', 'a') as f:
    f.write('lalala\n')
    
with open_file('test.log', 'r') as f:
    print(f.readlines())

['lalala\n']


In [41]:
class suppress_exception:
    def __init__(self, exc_type):
        self.exc_type = exc_type
    
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type == self.exc_type:
            print('Nothing happend.')
            return True
    
    
with suppress_exception(ZeroDivisionError) as f:
    print(type(f).__name__)
    big_number = 1 / 0


Nothing happend.


In [42]:
import contextlib


@contextlib.contextmanager
def printer(num):
    print('Enter')
    
    yield num ** 3
    
    print('Exit')

    
with printer(2) as p:
    print(p)

Enter
8
Exit


## Напишите контекстный менеджер который считает и выводит время, проведенное внутри него

In [45]:
import random
import time


class timer:
    def __init__(self):
        self.start = None
    
    def __enter__(self):
        self.start = time.time()
        return self
    
    def __exit__(self, *args):
        print('Elapsed — {}'.format(time.time() - self.start))


with timer():
    for _ in range(10):
        time.sleep(random.random())
        


Elapsed — 4.553982973098755


## Как добавить возможность получить текущее пройденное время?

In [121]:
import random
import time


class Timer:
    def __init__(self):
        self.start = time.time()
    
    @property
    def elapsed(self):
        return time.time() - self.start
    
    
class set_timer:
    def __init__(self):
        self.timer = Timer()
    
    def __enter__(self):
        return self.timer
    
    def __exit__(self, *args):
        print('Elapsed — {}'.format(self.timer.elapsed))
    

with set_timer() as t:
    for _ in range(10):
        time.sleep(random.random())
        print(t.elapsed)

0.14915704727172852
0.6862020492553711
1.433539867401123
1.914989948272705
2.6611268520355225
3.4602549076080322
4.229285001754761
4.950361013412476
4.9942498207092285
5.912122011184692
Elapsed — 5.912263870239258


# Асинхронные контекстные менеджеры

In [48]:
import aiohttp
import asyncio


async def fetch(client):
    async with client.get('http://python.org') as resp:
        assert resp.status == 200
        return await resp.text()


async def main(loop):
    async with aiohttp.ClientSession(loop=loop) as client:
        html = await fetch(client)
        print(html[:96])

        
loop = asyncio.get_event_loop()
loop.run_until_complete(main(loop))

<!doctype html>
<!--[if lt IE 7]>   <html class="no-js ie6 lt-ie7 lt-ie8 lt-ie9">   <![endif]-->


# Дескрипторы

In [120]:
class Descriptor:
    def __get__(self, obj, obj_type):
        print('get')
        
    def __set__(self, obj, value):
        print('set')


class Class:
    attr = Descriptor()
    

instance = Class()

instance.attr
instance.attr = 10

get
set


In [81]:
class Value:
    def __init__(self):
        self.value = None
    
    @staticmethod
    def _prepare_value(value):
        return value * 10

    def __get__(self, obj, obj_type):
        return self.value
    
    def __set__(self, obj, value):
        self.value = self._prepare_value(value)
    
    
class Class:
    attr = Value()

    
instance = Class()
instance.attr = 10

print(instance.attr)
print(Class.attr)

100
100


In [19]:
# Что происходит в момент доступа к атрибуту — instance.attr (Чтение)

def pseudo_read():
    if hasattr(instance.__class__, 'attr'):
        obj = instance.__class__.attr
        obj_type = obj.__class__

        if hasattr(obj_type, '__get__') and (
            hasattr(obj_type, '__set__') or 'attr' not in instance.__dict__
        ):
            return obj_type.__get__(obj, instance, instance.__class__)

    return instance.__dict__['attr']   


# 1. Data Descriptors
# 2. instance.__dict__
# 3. Non-Data Descriptors
# 4. class.__dict__
# 5. base classes


# instance.attr => type(instance).__dict__['attr'].__get__(instance, type(instance))
# Class.attr => Class.__dict__['attr'].__get__(None, Class)
    

In [25]:
# Что происходит во время — instance.attr = 10 (Записи)

def pseudo_write():
    if hasattr(instance.__class__, 'attr'):
        obj = instance.__class__.attr
        obj_type = obj.__class__

        if hasattr(obj_type, '__set__'):
            obj_type.__set__(obj, instance, 10)
            return

    instance.__dict__['attr'] = 10


## Напишите дескриптор, который пишет в переданный ему во время инициализации файл все присваиваемые значения

In [83]:
class PersistentField:
    def __init__(self, filename):
        self.filename = filename
        
    def __get__(self, obj, obj_type):
        return self.value
    
    def __set__(self, obj, value):
        with open(self.filename, 'a') as f:
            f.write(value)
            
        self.value = value


class Class:
    attr = PersistentField(filename='fast.db')
    
    
obj = Class()

obj.attr = 'Important string'

### Функции — это дескрипторы

In [84]:
class Class:
    def method(self):
        pass
    
    
obj = Class()    
print(obj.method)
print(Class.method)

<bound method Class.method of <__main__.Class object at 0x1020d8828>>
<function Class.method at 0x1020f8d08>


In [70]:
def summator(x, y):
    return x + y


lolwhat = summator.__get__(10)
# lolwhat(21)

### @classmethod & @staticmethod & @property — это дескрпиторы

In [119]:
class Property:
    def __init__(self, getter):
        self.getter = getter

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
    
        return self.getter(obj)
    
    
class Class:
    @property
    def x(self):
        return 'x'
    
    @Property
    def y(self):
        return 'y'
    
    def get_z(self):
        return 'z'
    
    z = Property(get_z)
    

obj = Class()

print(obj.x)
print(obj.y)
print(obj.z)

x
y
z


## Напишите реализацию @staticmethod или @classmethod

In [43]:
   
class Class:
    @ClassMethod
    def test(cls):
        print(cls.__name__)
    

a = Class()
a.test()


Class


In [118]:
class StaticMethod:
    def __init__(self, func):
        self.func = func
    
    def __get__(self, obj, obj_type):
        def wrapped(*args, **kwargs):
            return self.func(*args, **kwargs)
        
        return wrapped


class Class:
    @StaticMethod
    def test(message):
        print(message)
        
#     test = StaticMethod(test)
        

obj = Class()
obj.test('Hello')

Hello


# \_\_slots\_\_ (и здесь не без дескрипторов)

In [102]:
class Class:
    __slots__ = ['hello']
    
    def __init__(self):
        self.hello = 'test'

        
obj = Class()

# Мета-классы

### Классы — это объекты

In [112]:
class A:
    ...


In [111]:
a = A()

print(type(a))
print(type(A))

print(isinstance(a, A))
print(isinstance(A, type))

print(issubclass(A, type))
print(issubclass(A, object))

<class '__main__.A'>
<class 'type'>
True
True
False
True


![](https://blog.ionelmc.ro/2015/02/09/understanding-python-metaclasses/instance-of.png)


In [113]:
def dummy_factory():
    class Class:
        pass
    
    return Class


Dummy = dummy_factory()
obj = Dummy()

In [115]:
class Meta(type):
    def __new__(cls, name, parents, attrs):
        print('Creating {}'.format(name))
        if 'class_id' not in attrs:
            attrs['class_id'] = name.lower()
        
         return super().__new__(cls, name, parents, attrs)
    

class A(metaclass=Meta):
    pass


A.class_id

Creating A


'a'

In [116]:
class Meta(type):
    def __init__(cls, name, bases, attrs):
        print('Initializing — {}'.format(name))
        if not hasattr(cls, 'registry'):
            cls.registry = {}
        else:
            cls.registry[name.lower()] = cls
            
        super().__init__(name, bases, attrs)
        
        
class Base(metaclass=Meta):
    pass


class A(Base):
    pass


class B(Base):
    pass


Base.registry

Initializing — Base
Initializing — A
Initializing — B


{'a': __main__.A, 'b': __main__.B}

In [117]:
from abc import ABC, abstractmethod


class Abstract(ABC):
    @abstractmethod
    def abstract_method(self):
        pass


Abstract()

TypeError: Can't instantiate abstract class Abstract with abstract methods abstract_method

# Несколько полезных модулей стандартной библиотеки

### itertools

In [2]:
import itertools

In [24]:
# list(itertools.chain(range(10), range(10, 20)))

In [13]:
# list(itertools.combinations(range(5), 3))

In [ ]:
# list(itertools.permutations(range(3), 3))

In [5]:
# g = itertools.cycle('ABC')
# for _ in range(10):
#     print(next(g))

In [8]:
for number in itertools.count():
    if number > 10:
        break
    
#    print(number)

In [22]:
# list(itertools.repeat(1, 10))

In [16]:
# list(itertools.product('ABC', 'xy'))

In [20]:
# list(itertools.zip_longest(range(10), range(5), fillvalue=None))

### re

In [46]:
import re

In [56]:
# sentence = 'Call me ASAP!!!11 8-990-1122239 srsly.'
# re.search(r' ([\d-]+) ', sentence).groups()

In [121]:
# re.findall(r"(\w+)", "Python is the capital of Great Britain")

In [120]:
# re.search(r"(123)", "a123zzb").group(1)

In [51]:
# re.sub('(\.)', ' :)', 'Hello...')

In [49]:
# ints = re.compile(r"(\d+)")
# re.findall(ints, "a123zaa223zb")

### collections

In [31]:
import collections

In [40]:
colors = ['red', 'blue', 'red', 'green', 'blue', 'blue']
counter = collections.Counter(colors)

# print(counter)

In [43]:
# print(counter.most_common(2))

In [88]:
d = collections.defaultdict(lambda: 'not here')   
# print(d['missing_key'])

In [89]:
# collections.OrderedDict()

In [69]:
class SuperList(collections.abc.MutableSequence):
    pass


### datetime

In [91]:
from datetime import datetime, timedelta

In [108]:
midterm = datetime.now() + timedelta(days=7)
# print(midterm.day)

In [101]:
# print(datetime.now().strftime('%Y / %m / %d'))

In [104]:
# print(datetime.strptime('2016-10-25', '%Y-%m-%d'))